# Identification of Fall Event Records in Clinical Notes using `fastText`

## Install and Import packages

In [ ]:
!pip install fasttext

In [ ]:
import fasttext
import pandas as pd

fasttext.FastText.eprint = lambda x: None

## Load Datasets

In [ ]:
TRAIN_FILE_NAME = "train_sets_9538_20231"
TEST_FILE_NAME = "test_sets_565_14249"

In [ ]:
train_df = pd.read_csv(f'{TRAIN_FILE_NAME}.csv', encoding='utf8')

In [ ]:
test_df = pd.read_csv(f'{TEST_FILE_NAME}.csv', encoding='utf8')

In [ ]:
dataType = ["anonym", "#ProcessedData", "#TokenizedData", "translation"]

### Check the Rate of Falls in Datasets

In [ ]:
X_train = train_df[[f"{dataType[0]}"]]
y_train = train_df['label']

X_test = test_df[[f"{dataType[0]}"]]
y_test = test_df['label']


print('----------- Rate of Falls in Train Datasets -----------')
print(f'Non-Fall Data (label : 0) = {round(y_train.value_counts()[0]/len(y_train) * 100,3)}%')
print(f'    Fall Data (label : 1) = {round(y_train.value_counts()[1]/len(y_train) * 100,3)}%')
print()
print('----------- Rate of Falls in Test Datasets -----------')
print(f'Non-Fall Data (label : 0) = {round(y_test.value_counts()[0]/len(y_test) * 100,3)}%')
print(f'    Fall Data (label : 1) = {round(y_test.value_counts()[1]/len(y_test) * 100,3)}%')

## Pre-Processing

### Remove CRLF

In [ ]:
def remove_CRLF(input_str):
  input_str = input_str.replace("\n", " ")
  input_str = input_str.replace("\\n", " ")
  input_str = input_str.replace("\r", " ")
  input_str = input_str.replace("\\r", " ")

  return input_str

In [ ]:
for dataName in dataType:
  for i in range(len(train_df)):
    train_df[f"{dataName}"][i] = remove_CRLF(train_df[f"{dataName}"][i])
  for j in range(len(test_df)):
    test_df[f"{dataName}"][j] = remove_CRLF(test_df[f"{dataName}"][j])

### Generate `.txt` files for training `fastText` model

In [ ]:
for dataName in dataType:  
  # train
  ft_train_df = train_df.copy()
  for i in range(len(ft_train_df)):
    ft_train_df['label'][i] = "__label__" + str(ft_train_df['label'][i])
  ft_train_df = ft_train_df[['label', f"{dataName}"]]
  ft_train_df.to_csv(f"{TRAIN_FILE_NAME}_{dataName}.txt", index=False, header=None, sep="\t")
  
  # test
  ft_test_df = test_df.copy()
  for i in range(len(ft_test_df)):
    ft_test_df['label'][i] = "__label__" + str(ft_test_df['label'][i])
  ft_test_df = ft_test_df[['label', f"{dataName}"]]
  ft_test_df.to_csv(f"{TEST_FILE_NAME}_{dataName}.txt", index=False, header=None, sep="\t")

## Training Model

In [ ]:
for dataName in dataType:
    model = fasttext.train_supervised(input=f"{TRAIN_FILE_NAME}_{dataName}.txt",
                                      epoch=100,
                                      lr = 1,
                                      wordNgrams=2,
                                      dim=300,
                                      )
    
    # Saving Model
    model.save_model(f"fastText_Model_{TRAIN_FILE_NAME}_{dataName}.bin")

## Testing Model

In [ ]:
from sklearn.metrics import precision_score, recall_score, f1_score, accuracy_score, confusion_matrix

def print_result(dataName, y_test, predicted):
    print(f"------ Data Type : {dataName} ------")
    print('Precision        : %.4f' % precision_score(y_test, predicted))
    print('Recall           : %.4f' % recall_score(y_test, predicted))
    print('F1 score         : %.4f' % f1_score(y_test, predicted))
    print('Accuracy score   : %.4f' % accuracy_score(y_test, predicted))
    print('Confusion Matrix :')
    print(confusion_matrix(y_test, predicted))
    print()

In [ ]:
for dataName in dataType:
    # Load saved model
    model = fasttext.load_model(f"fastText_Model_{TRAIN_FILE_NAME}_{dataName}.bin")
  
    predicted = []
    for test in test_df[f"{dataName}"]:
      predicted.append(int(model.predict(test, k=1)[0][0][9:]))

    # Print test result
    print_result(dataName, y_test, predicted)

---
## Option) `zip` the entire file

In [ ]:
!zip datasets_and_fastText_model.zip *